SMT version for the MCP problem given in the Combinatorial Decision Making and Optimization course.

The model is based on the one already developed for the CP version of the problem with the necessary modifications to make it work for the SMT version.

Necessary libraries:

In [14]:
#!pip install ortools

Necessary imports:

In [15]:
from ortools.linear_solver import pywraplp
import matplotlib.pyplot as plt # Matplotlib for plotting
import numpy as np # Numpy for array manipulation
import time
import json

In [16]:
def solve_courier_problem(m, n, courier_capacities, item_sizes, item_distances):
    num_couriers = m
    num_items = n
    # I save the starting time
    st = time.time()
    
    # I create the SOLVER
    solver = pywraplp.Solver.CreateSolver('SCIP')
    
    # I create the VARIABLES
    # Maximum distance travelled by a courier
    max_distance = solver.NumVar(0, solver.infinity(), 'max_distance')
    # Assignment matrix where assignment[i][j][k] is 1 if courier i takes item j and item k going from j to k
    assignment = [[[solver.IntVar(0, 1, f'assignment_{i}_{j}_{k}') for j in range(num_items + 1)] for k in range(num_items + 1)] for i in range(num_couriers)]
    # Matrix for the MTZ subtours-breaking formulation, where u[i][j] is the position of the node j in the tour of courier i
    u = [[solver.IntVar(0, solver.infinity(), f'u_{i}_{j}') for j in range(num_items + 1)] for i in range(num_couriers)]
    
    # I create the CONSTRAINTS
    # The diagonal of the matrix is 0
    solver.Add(sum(assignment[i][j][j] for i in range(num_couriers) for j in range(num_items + 1)) == 0)
    # Items are assigned to at most one courier
    for j in range(num_items):
        solver.Add(sum(assignment[i][j][k] for i in range(num_couriers) for k in range(num_items + 1)) == 1)
        solver.Add(sum(assignment[i][k][j] for i in range(num_couriers) for k in range(num_items + 1)) == 1)
    # Create multiple constraints
    for i in range(num_couriers):
        # We set lower boundaries for max distance, so that we can minimize it later
        solver.Add(sum(assignment[i][j][k] * item_distances[j][k] for j in range(num_items + 1) for k in range(num_items + 1)) <= max_distance)
        # Each courier carries at least one item
        solver.Add(sum(assignment[i][num_items][k] for k in range(num_items + 1)) == 1)
        # Courier capacities are respected
        solver.Add(sum(assignment[i][j][k] * item_sizes[j] for j in range(num_items) for k in range(num_items + 1)) <= courier_capacities[i])
        for j in range(num_items + 1):
            # Create constraint: n arcs in, n arcs out
            solver.Add(sum(assignment[i][j][k] for k in range(num_items + 1)) == sum(assignment[i][k][j] for k in range(num_items + 1)))

    for i in range(num_couriers):
        for j in range(0, num_items + 1):
            for k in range(0, num_items + 1):
                if j != k and j != num_items:
                    # I add the constraint of the MTZ formulation such that the courier goes forward and visits all the nodes assigned to him
                    solver.Add(u[i][j] - u[i][k] + 1 <= (num_items - 1) * (1 - assignment[i][j][k]))
    
    solver.Minimize(max_distance)
    
    solver.set_time_limit(300000)
    
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        for i in range(num_couriers):
            print(f'Courier {i} takes item: ')
            print('{')
            for j in range(num_items):
                for k in range(num_items + 1):
                    if assignment[i][j][k].solution_value() > 0:
                        print(f' -> {j}')
            print('}')
        print('Max distance:', max_distance.solution_value())
        print('Objective value:', solver.Objective().Value())
        # I print the values of assignment
        # for each courier i create and print a item matrix
        print()
        for i in range(num_couriers):
            print(f'Courier {i} item matrix:')
            for j in range(num_items + 1):
                for k in range(num_items + 1):
                    print(int(assignment[i][j][k].solution_value()), end=' ')
                print()
            print()
        # I print the values of u
        print()
        print('u matrix:')
        for i in range(num_couriers):
            for j in range(num_items + 1):
                print(int(u[i][j].solution_value()), end=' ')
            print()
        print(item_distances)
        et = int(time.time())
        execution_time = et - st
        # number of items per courier
        n_items_per_courier = []
        # coordinates of items per courier
        coordinates_per_courier = []
        # for each courier i create a list of coordinates and a counter of items
        for i in range(num_couriers):
            coordinates_per_courier.append(dict())
            n_items_per_courier.append(0)
            for j in range(num_items + 1):
                for k in range(num_items):
                    if assignment[i][j][k].solution_value() > 0:
                        n_items_per_courier[i] += 1
                        # I save the coordinates of the item in the dictionary, where the key is the starting node and the value is the ending node
                        coordinates_per_courier[i][j] = k
        print(n_items_per_courier)
        print(coordinates_per_courier)
        best_paths_items = [[] for i in range(num_couriers)]
        # for each courier i create a list of items from the dictionary but in a ordered, formated fashion
        for i in range(num_couriers):
            best_paths_items[i].append(coordinates_per_courier[i][num_items])
            while len(best_paths_items[i]) < n_items_per_courier[i]:
                best_paths_items[i].append(coordinates_per_courier[i][best_paths_items[i][-1]])
        print(best_paths_items)
                    
        return max_distance.solution_value(), best_paths_items, execution_time
    else:
        print('The problem does not have an optimal solution.')
        return -1, [[]], -1

The variable instances (like number of couriers) are defined in a .dat file. The file is read and the variables are defined.

In [20]:
def printJson():

    solverName = 'ortools'
    numInstances = 15
    formatted_list = ["%.2d" % i for i in range(1, numInstances + 1)]
    simple_list = [i for i in range(1, numInstances + 1)]
   


    for i in range(11,numInstances):
         # open the file in Instances folder
        f = open(f"Instances/inst{formatted_list[i]}.dat", "r")
        # the first line is the number of couriers
        m = int(f.readline())
        # the second line is the number of items
        n = int(f.readline())
        # the third line is the load size of each courier
        load_size = [int(x) for x in f.readline().split()]
        # the fourth line is the size of each item
        item_size = [int(x) for x in f.readline().split()]
        # the rest is the distance matrix
        distance = []
        for j in range(n+1):
            distance.append([int(x) for x in f.readline().split()])
        # close the file
        f.close()

        print("Solving Instance", i+1, "with ortools")
        obj, sol, time = solve_courier_problem(m,n, load_size, item_size, distance)
        opt = True

        #adding 1 to all the elements of sol
        for j in range(len(sol)):
            sol[j] = np.array(sol[j]) + 1
            sol[j] = sol[j].tolist()



        data = dict([(solverName,dict([
            ('time', time),
            ('optimal',opt),
            ('obj', obj),
            ('sol', sol)
            ]))])


        json_string = json.dumps(data)

        with open(f'./output_folder/MIP/{simple_list[i]}.json', 'w') as outfile:
            outfile.write(json_string)


In [21]:
printJson()

Solving Instance 12 with ortools
The problem does not have an optimal solution.
Solving Instance 13 with ortools
The problem does not have an optimal solution.
Solving Instance 14 with ortools
The problem does not have an optimal solution.
Solving Instance 15 with ortools
The problem does not have an optimal solution.
